In [5]:
""" Auto-number FRS items from formatted list
*+FRS1234+*
#
##
###
*** Non-numbered FRS text or sub-item
#
##
###

becomes 

*+FRS1234+*
* *FRS1234.1:*
** *FRS1234.1.1:*
*** *FRS1234.1.1.1:*
**** Non-numbered FRS text or sub-item

* *FRS1234.2:*
** *FRS1234.2.1:*
*** *FRS1234.2.1.1:* 
"""


' Auto-number FRS items from formatted list\n*+FRS1234+*\n#\n##\n###\n*** Non-numbered FRS text or sub-item\n#\n##\n###\n\nbecomes \n\n*+FRS1234+*\n* *FRS1234.1:*\n** *FRS1234.1.1:*\n*** *FRS1234.1.1.1:*\n**** Non-numbered FRS text or sub-item\n\n* *FRS1234.2:*\n** *FRS1234.2.1:*\n*** *FRS1234.2.1.1:* \n'

In [12]:
import re
subitem_match = re.compile(r"^(?:([#]+)\s*)").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match
header_match = re.compile(r"^(?:([\*\#]*) )?([\+\*]+)([^\*\+]+?)\:?([\+\*]+)\:?\s*(.+)?$").match
frs_match = re.compile(r"([FU]RS\d*)\.?([\d\.]+)?").match
num_escape_match = re.compile(r"^(\*{2})(\#+)").match
image_caption_match = re.compile(r"(Image \d+)").match
toplevel_match = re.compile(r"^([\+\*]+)([^\*\+]+?)\:?([\+\*]+)\:?\s*(.+)?$").match

class FRSNumberStateMachine():
    VALID_INDENTS = "*+"
    
    # inputs
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    LINE_HEADER = 3
    LINE_NUM_ESCAPE = 4
    LINE_IMG_CAPTION = 5
    
    # states
    ST_INIT = 0
    ST_NONUM = 1
    ST_FRS = 2
    
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = []
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
        self.sm_st = self.ST_INIT
        self.sm = [
            # ST_INIT             ST_NONUM             ST_FRS
            [self.set_frs,        self.set_frs,        self.set_frs],        # LINE_FRS
            [None,                None,                self.set_subitem],    # LINE_SUBITEM
            [None,                None,                None],                # LINE_OTHER
            [self.set_header,     self.set_header,     self.set_header],     # LINE_HEADER
            [self.set_num_escape, self.set_num_escape, self.set_num_escape], # LINE_NUM_ESCAPE
            [None,                None,                None],                # LINE_IMG_CAPTION
        ]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
        
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
    
    def format_lvl(self, lvl):
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def feed(self, line):
        ev, arg = self._parse_input(line)
        func = self.sm[ev][self.sm_st]
        if func:
            st, rv = func(arg)
            self.sm_st = st
        else:
            rv = line
        return rv
    
    def _parse_input(self, line): 
        """Parse the input line to determine the 
        type of input, and any arguments to be passed
        to the input event handler.
        """
        m = header_match(line)
        if m:
            ind, fmt, s, fmt2, _ = m.groups()

            # Verify format to avoid false positive
            # if other * or + characters appear in the string
            if fmt == fmt2[::-1]:
                m2 = frs_match(s)
                if m2:
                    typ = self.LINE_FRS
                    frs, nums = m2.groups()
                    arg = (line, ind, fmt, frs, nums)
                    return typ, arg
                if image_caption_match(s):
                    return self.LINE_IMG_CAPTION, ()
                else:
                    typ = self.LINE_HEADER
                    arg = (line, ind, fmt, s)
                return typ, arg
        m = subitem_match(line)
        if m:
            typ = self.LINE_SUBITEM
            arg = line, len(m.group(1) or "")
            return typ, arg
        m = num_escape_match(line)
        if m:
            typ = self.LINE_NUM_ESCAPE
            arg = line,
            return typ, arg
        return self.LINE_OTHER, (line,)
    
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def parse_text(self, text):
        for line in text.splitlines():
            yield self.feed(line)
    
    # State machine methods
                                
    def set_frs(self, arg):
        line, indent, fmt, frs, nums = arg
        self.ilvl = len(indent or "")
        self.fmt = fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        return self.ST_FRS, line
    
    def set_header(self, arg):
        return self.ST_NONUM, arg[0]
    
    def set_subitem(self, arg):
        line, n = arg
        self.set_indent(n)
        fmt = self.frs_string()
        try:
            _, tail = line.split(" ", 1)
        except ValueError:
            v = fmt
        else:
            v = " ".join((fmt, tail))
        return self.sm_st, v
    
    def set_num_escape(self, arg):
        line, = arg
        return self.sm_st, line.lstrip("*")
    
NumState = FRSNumberStateMachine

In [16]:
ntests = 0
passed = 0
errors = []
def assert_ab(a, b, equal=True):
    global ntests, passed, errors
    ntests += 1
    try:
        if equal:
            assert a == b, "%r != %r" % (a, b)
        else:
            assert a != b, "%r == %r" % (a, b)
    except AssertionError as e:
        val = str(e)
        if isinstance(a, str) and isinstance(b, str):
            val += "\n" + str(a.splitlines()) + "\n" + str(b.splitlines())
        errors.append(val)
    else:
        passed += 1

def assert_header_match(s, exp):
    assert_ab(header_match(s).groups(), exp)
        
assert header_match("**") == None
assert header_match("***") == None
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("*URS*", (None, "*", "URS", "*", None))
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*", ("*", "*", "FRS1234", "*", None))
assert_header_match("*FRS1234.5.6*", (None, "*", "FRS1234.5.6", "*", None))
assert_header_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234.5.6", "+*", None))
assert_header_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234.5.6.7.8", "+*", None))
assert_header_match("* *FRS1234:*", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*:", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", "*", "Bob"))



def assert_num_escape(s, exp):
    m = num_escape_match(s)
    r = m.groups() if m else None
    assert_ab(r, exp)
    
assert_num_escape("**# Bob", ("**", "#"))
assert_num_escape("**## Bob", ("**", "##"))
assert_num_escape("**### Bob", ("**", "###"))
assert_num_escape("# Bob", None)
assert_num_escape("## Bob", None)
assert_num_escape("### Bob", None)
assert_num_escape("* Bob", None)
assert_num_escape("** Bob", None)
assert_num_escape("*** Bob", None)

#def assert_img_match(s):
#    m = image_match(s)
#    assert_ab(bool(m), True)
    
#assert_img_match("Image 1")

def set_str(st, s):
    ind, fmt, fs, _, _ = header_match(s).groups()
    frs, nums = frs_match(fs).groups()
    st.set_frs((s, ind, fmt, frs, nums))
    

def test_state(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1*", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1*", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    set_str(st, s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    set_str(st, s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

def assert_feed_indent(s, feed, exp):
    st = NumState()
    set_str(st, s)
    st.sm_st = st.ST_FRS
    st.feed(feed)
    assert_ab(st.frs_string(), exp)
    
assert_feed_indent("*FRS1234*", "# Bob", "* *FRS1234.1:*")
assert_feed_indent("*FRS1234*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "# Bob", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## Bob", "** *FRS1234.1.1.1:*")
assert_feed_indent("* *FRS1234.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("* *FRS1234*", "## Bob", "** *FRS1234.1:*")
assert_feed_indent("** *FRS1234.1*", "## Bob", "** *FRS1234.2:*")
assert_feed_indent("* *FRS1234.1*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("** *FRS1234.1.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("*FRS1234.1*", "# ", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## ", "** *FRS1234.1.1.1:*")
assert_feed_indent("*FRS1234.1*", "#", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "##", "** *FRS1234.1.1.1:*")

def assert_double_parse(t, d=5):
    r = list(NumState(d).parse_text(t))
    r2 = list(NumState(d).parse_text("\n".join(r)))
    assert_ab(r, r2)
    
dpt = """*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""
assert_double_parse(dpt)

def assert_new_state(s, exp, nxt=1, ind=0):
    st = NumState()
    for l in s.splitlines():
        st.feed(l)
    for _ in range(nxt):
        st.next()
    for _ in range(ind):
        st.indent()
    assert_ab(st.frs_string(), exp)
    
assert_new_state("*FRS123.1*\n*FRS31.2*", "*FRS31.3:*")
assert_new_state("*FRS123.1*\n*FRS123.3*", "*FRS123.4:*")

def assert_parse_text(txt, exp):
    st = NumState()
    lines = list(st.parse_text(txt))
    res = "\n".join(lines)
    assert_ab(res, exp)

lines = """*FRS1234.1:*
#"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*"""
assert_parse_text(lines, exp)

lines="""*FRS1234.1:*
#
## Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
#
##
"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*
** *FRS1234.1.1.1:* Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
* *FRS1234.2.1:*
** *FRS1234.2.1.1:*"""

assert_parse_text(lines, exp)

lines = "+*FRS*+\n*FRS1234:*\n#"
exp = "+*FRS*+\n*FRS1234:*\n* *FRS1234.1:*"
assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
*** Bob
## Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
*** Bob
** *FRS2908.1.1.1:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
#### Biff
*** Bob
# Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
**** *FRS2908.1.1.1.1.1:* Biff
*** Bob
* *FRS2908.1.2:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.0*
# Bob1
## Bob2
*** Fred1
*** Fred2
## Bob3
## Bob4
## Bob5
*** Fred3
#### Bob6
#### Bob7
#### Bob8
*** Fred4
# Bob9
"""

exp = """*FRS2908.0*
* *FRS2908.0.1:* Bob1
** *FRS2908.0.1.1:* Bob2
*** Fred1
*** Fred2
** *FRS2908.0.1.2:* Bob3
** *FRS2908.0.1.3:* Bob4
** *FRS2908.0.1.4:* Bob5
*** Fred3
**** *FRS2908.0.1.4.1.1:* Bob6
**** *FRS2908.0.1.4.1.2:* Bob7
**** *FRS2908.0.1.4.1.3:* Bob8
*** Fred4
* *FRS2908.0.2:* Bob9"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** Fred:
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** Fred:
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** considered +/- 15 degrees
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** considered +/- 15 degrees
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*FRS123* 
# Bob
*Image 1*: Bob's image
# Bob"""

exp = """*FRS123* 
* *FRS123.1:* Bob
*Image 1*: Bob's image
* *FRS123.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*URS2908.0* 
# Bob
** considered +/- 15 degrees
# Bob"""

exp = """*URS2908.0* 
* *URS2908.0.1:* Bob
** considered +/- 15 degrees
* *URS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

def assert_parse_empty(s, exp):
    st = NumState()
    res = None
    for res in st.parse_text(s):
        pass
    assert_ab(res, exp)
    
assert_parse_empty("*User Requirements*\n# Bar", "# Bar")

def finish():
    print("%d/%d tests passed successfully" % (passed, ntests))
    for e in errors:
        print(e)
    if errors:
        raise Exception("Unit testing suite failed")
finish()

95/95 tests passed successfully


In [14]:
toplevel_match = re.compile(r"^([\+\*]+)([^\*\+]+?)\:?([\+\*]+)\:?\s*(.+)?$").match
def assert_ab2(a, b, equal=True):
    if equal:
        assert a == b, "%r != %r" % (a, b)
    else:
        assert a != b, "%r == %r" % (a, b)
    
            
def assert_toplevel_match(s, exp):
    assert_ab(toplevel_match(s).groups(), exp)
assert_ab2(toplevel_match("Test"), None)
assert_ab2(toplevel_match("* *+Test Test+*"), None)

AssertionError: <_sre.SRE_Match object; span=(0, 15), match='* *+Test Test+*'> != None

In [17]:
import clipboard

In [25]:
template=  r'''lines=r"""%s
"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)
'''
s = template % clipboard.paste()
clipboard.copy(s)
exec(s)

*+Background+*
This request began as a set of features to be used for the IQ/OQ process, and includes some features that would be useful during that process which may also be useful for customers for other purposes. 

Ultimately, the desired result is to simplify the process of sharing configurations between multiple bioreactors, particularly on GMP configured machines. On these units, config files must be either transferred by hand (slow and error-prone) or they must be configured by a Windows account admin (requiring IT support). Currently, only a customer's IT personnel are expected to have access to the Windows admin account rather than process administrators. Simplifying this process would reduce error rates and make it easier for customers to ensure configuration uniformity across multiple bioreactors. 

For example, a customer may have units A and B with custom configurations for Logger Settings, Alarms, and System Variables. To transfer these settings to unit C, they have to ma

In [22]:
lines=r"""*+Background+*
This request began as a set of features to be used for the IQ/OQ process, and includes some features that would be useful during that process which may also be useful for customers for other purposes. Ultimately, the desired result is to simplify the process of sharing configurations between multiple bioreactors, particularly on GMP configured machines. On these units, config files must be either transferred by hand, which is slow and error-prone, or they must be configured by a Windows account admin. Currently, only a customer's IT personnel are expected to have access to the Windows admin account. Simplifying this process would reduce error rates and make it easier for customers to ensure configuration uniformity across multiple bioreactors. 

For example, a customer may have units A and B with custom configurations for Logger Settings, Alarms, and System Variables. To transfer these settings to unit C, they have to manually transfer the files to the correct locations, reboot the RIO, reboot the Shell UI, and ensure that they didn't do anything in the wrong order that might cause the files to be accidentally overridden. A builtin mechanism to perform this process would greatly simplify it. 

This has happened to some customers before (CDI? Semma?), who attempted to transfer config files between units of different sizes, and ended up screwing up their settings badly enough to require hours worth of correspondence with engineers at PBS to resolve the issue. 

This initial set of feature requests is intended to be simple and straightforward. Future improvements can incrementally build on this system to include features such as sanity checks, file bundles, partial files, or others. 

Some of the URS/FRS items below are labeled with "(Low)" or "(Med)", indicating they are not strictly necessary for the features to be used by customers, but would be convenient and should be included if it doesn't take a significant amount of time to implement or test. 

Other features to be implemented include using the recorder.exe to save factory defaults, and providing a facility to restore config files to factory defaults. Among other things, this is a much easier way of allowing customers to restore defaults in the case of an issue than digging through the database. While the database method fulfills the purpose of providing a permanent record, its much easier to just have the actual file there and ready to use. 

*+URS+*
# During the manufacturing process, in IM00226 and IM00211, recorder.exe should back up all config files to a read-only folder. 
# There should be a method present in the Desktop UI to back up current config files from LabVIEW Data and RIO to a designated folder. 
## The backup folder should be accessible from the file browser tab, so that it can be transferred to a network drive or USB. 
## (Low) Users should be able to select which files to backup and the name of the folder to backup to. 
# There should be a method to import config files from either USB or a backup folder as created by recorder.exe.  
## Config files should be checked for structure before being imported
## Appropriate permissions should be checked to ensure that an unauthorized user cannot override files. 


*+FRS+*
# A new menu will be added, accessible from a new button present in the "System" dropdown menu. 
## (Med) The user should be able to enter a folder 
## (Low) The User should be able to select which files to backup. 
## (Low) The menu should include select all and deselect all buttons. 
"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)


*+Background+*
This request began as a set of features to be used for the IQ/OQ process, and includes some features that would be useful during that process which may also be useful for customers for other purposes. Ultimately, the desired result is to simplify the process of sharing configurations between multiple bioreactors, particularly on GMP configured machines. On these units, config files must be either transferred by hand, which is slow and error-prone, or they must be configured by a Windows account admin. Currently, only a customer's IT personnel are expected to have access to the Windows admin account. Simplifying this process would reduce error rates and make it easier for customers to ensure configuration uniformity across multiple bioreactors. 

For example, a customer may have units A and B with custom configurations for Logger Settings, Alarms, and System Variables. To transfer these settings to unit C, they have to manually transfer the files to the correct locations,